<a href="https://www.kaggle.com/code/kentvejrupmadsen/classification-of-letters?scriptVersionId=133582928" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [1]:
%pip install ipykernel
!python -m ipykernel install --user --name=notebook_environment

%pip install matplotlib
%pip install keras
%pip install tensorflow
%pip install kaggle
%pip install numpy
%pip install wandb

Note: you may need to restart the kernel to use updated packages.
Installed kernelspec notebook_environment in /root/.local/share/jupyter/kernels/notebook_environment
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [2]:
from random \
    import SystemRandom

import tensorflow

from tensorflow.data \
    import AUTOTUNE

from kaggle_secrets \
    import UserSecretsClient

from keras.losses \
    import SparseCategoricalCrossentropy

from keras.utils \
    import image_dataset_from_directory

from keras \
    import \
    Model, \
    Sequential

from keras \
    import layers

import wandb

from wandb.integration.keras \
    import \
    WandbCallback, \
    WandbEvalCallback, \
    WandbMetricsLogger, \
    WandbModelCheckpoint

/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:98: UserWarning: unable to load libtensorflow_io_plugins.so: unable to open file: libtensorflow_io_plugins.so, from paths: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so']
caused by: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so: undefined symbol: _ZN3tsl6StatusC1EN10tensorflow5error4CodeESt17basic_string_viewIcSt11char_traitsIcEENS_14SourceLocationE']
  warnings.warn(f"unable to load libtensorflow_io_plugins.so: {e}")
/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:104: UserWarning: file system plugins are not loaded: unable to open file: libtensorflow_io.so, from paths: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io.so']
caused by: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io.so: undefined symbol: _ZTVN10tenso

In [3]:
secrets = UserSecretsClient()
random_generator = SystemRandom()

In [4]:
# Constants
def label_training() -> str:
    return 'training'

def label_validation() -> str:
    return 'validation'

def empty_string() -> str:
    return ''

def zero() -> int:
    return 0

def get_log_runs() -> bool:
    global log_runs
    return log_runs

def get_gpus_to_use() -> list:
    return [ '/gpu:0', '/gpu:1' ]

# Dynamic
def set_size_of_training_set_categories(
    value: int
) -> None:
    global configuration
    configuration['dataset']['training']['size'] = value

def set_labels_of_training_set(
    value: list
) -> None:
    global configuration
    configuration['dataset']['training']['labels'] = value
    set_size_of_training_set_categories(
        len(
            value
        )
    )

def get_size_of_training_set_categories() -> int:
    global configuration
    return configuration['dataset']['training']['size']



def set_size_of_validation_set_categories(
    value: int
) -> None:
    global configuration
    configuration['dataset']['validation']['size'] = value


def set_labels_of_validation_set(
    value: list
) -> None:
    global configuration
    configuration['dataset']['validation']['labels'] = value
    
    set_size_of_validation_set_categories(
        len(
            value
        )
    )


def get_size_of_validation_set_categories() -> int:
    global configuration
    return configuration['dataset']['validation']['size']

def get_max_random_value() -> int:
    return 16777215

def get_min_random_value() -> int:
    return zero()

def generate_random_seed() -> int:
    global random_generator
    
    return random_generator.randint(
        get_min_random_value(), 
        get_max_random_value()
    )

In [5]:
__wandb_key: str = secrets.get_secret('__wandb__')

def get_wandb_secret() -> str:
    global __wandb_key
    
    if __wandb_key is None:
        return empty_string()
    
    return __wandb_key


In [6]:
def generate_config_for_a_dataset() -> dict:
    return {
        'labels': [],
        'size': zero(),
        'seed': generate_random_seed(),
        'shuffle': True,
        'crop': False
    }

def generate_config_view() -> dict:
    return {
        'height': 512,
        'width': 512,
        'channels': 3
    }

def generate_config_datasets() -> dict:
    return {
        'split_validation_at': 0.45,
        'training': generate_config_for_a_dataset(),
        'validation': generate_config_for_a_dataset()
    }

def generate_config() -> dict:
    return {
        'batch_size': 12,
        'epochs': 10,

        'view': generate_config_view(),
        'dataset': generate_config_datasets(),
    }

configuration = generate_config()

In [7]:
def get_configuration() -> dict:
    global configuration
    return configuration

def get_configuration_by_key(
    key: str
):
    return get_configuration()[key]

def get_configuration_in_view_by_key(
    key: str
):
    return get_configuration_by_key('view')[key]

def get_configuration_in_dataset_by_key(
    key: str
):
    return get_configuration_by_key('dataset')[key]

def get_configuration_in_training_set_by_key(
    key: str
):
    return get_configuration_in_dataset_by_key('training')[key]

def get_configuration_in_validation_set_by_key(
    key: str
):
    return get_configuration_in_dataset_by_key('validation')[key]


In [8]:
#
use_cache: bool = False
log_runs: bool = True

cache_buffer_size: int = AUTOTUNE
epochs = get_configuration_by_key('epochs')
dataset_batch_size: int = get_configuration_by_key('batch_size')

vision_height: int = get_configuration_in_view_by_key('height')
vision_width: int = get_configuration_in_view_by_key('width')

vision_number_of_color_channels: int = get_configuration_in_view_by_key('channels')

vision_size = (
    vision_height, 
    vision_width
)

vision_size_w_channels = (
    vision_height, 
    vision_width, 
    vision_number_of_color_channels
)

split_dataset_at: float = get_configuration_in_dataset_by_key('split_validation_at')

path_to_model: str = '/kaggle/working/model'
path_to_dataset: str = '/kaggle/input/letter-images-dataset/dataset'

training_dataset = None
validation_dataset = None

pointer_to_training_dataset = None
pointer_to_validation_dataset = None

model = None
    
# Callbacks
wandb_callback = None

fit_callbacks = []

# result
results: list = []

strategy = tensorflow.distribute.MirroredStrategy(
    devices=get_gpus_to_use()
)

In [9]:
def start_process() -> None:
    if get_log_runs():
        wandb.login(
            key=get_wandb_secret()
        )
        
def initialise_process() -> None:
    if get_log_runs():
        wandb.init(
            project = 'letter-identification',
            entity = 'designermadsen',    
            config = configuration,
            sync_tensorboard = True,
            save_code = True,
            monitor_gym = True
        )

def end_process() -> None:
    if get_log_runs():
        wandb.finish()


In [10]:
start_process()

training_dataset = image_dataset_from_directory(
    path_to_dataset,
    validation_split = split_dataset_at,
    subset= label_training(),
    seed = get_configuration_in_training_set_by_key('seed'),
    image_size = vision_size,
    batch_size = dataset_batch_size,
    shuffle = True,
    crop_to_aspect_ratio = True
)

set_labels_of_training_set(
    training_dataset.class_names
)

validation_dataset = image_dataset_from_directory(
    path_to_dataset,
    validation_split = split_dataset_at,
    subset = label_validation(),
    seed = get_configuration_in_validation_set_by_key('seed'),
    image_size = vision_size,
    batch_size = dataset_batch_size,
    shuffle = True,
    crop_to_aspect_ratio = True
)

set_labels_of_validation_set(
    validation_dataset.class_names
)

initialise_process()

wandb: W&B API key is configured. Use `wandb login --relogin` to force relogin
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


Found 11339 files belonging to 67 classes.
Using 6237 files for training.
Found 11339 files belonging to 67 classes.
Using 5102 files for validation.


wandb: Currently logged in as: designermadsen. Use `wandb login --relogin` to force relogin
wandb: wandb version 0.15.4 is available!  To upgrade, please run:
wandb:  $ pip install wandb --upgrade
wandb: Tracking run with wandb version 0.15.3
wandb: Run data is saved locally in /kaggle/working/wandb/run-20230614_160110-cl312tzo
wandb: Run `wandb offline` to turn off syncing.
wandb: Syncing run woven-jazz-20
wandb: ⭐️ View project at https://wandb.ai/designermadsen/letter-identification
wandb: 🚀 View run at https://wandb.ai/designermadsen/letter-identification/runs/cl312tzo


In [11]:
def generate_input_layers(
    ml_layers: list
) -> list:
    global vision_size_w_channels
    
    ml_layers.append(
        layers.Rescaling(
            1./255, 
            input_shape=vision_size_w_channels, 
            trainable=True
        )
    )
    
    return ml_layers


In [12]:
def generate_middle_layers(
    ml_layers: list
) -> list:
    merge_layer = (2, 2)
    
    ml_layers.append(
        layers.Conv2D(
            256, 
            3, 
            padding='same', 
            activation='relu'
        )
    )
    
    ml_layers.append(
        layers.Conv2D(
            256, 
            3, 
            padding='same', 
            activation='relu'
        )
    )
    
    ml_layers.append(
        layers.MaxPooling2D(
            pool_size=merge_layer
        )
    )
    
    ml_layers.append(
        layers.Conv2D(
            128, 
            3, 
            padding='same', 
            activation='relu'
        )
    )
    
    ml_layers.append(
        layers.MaxPooling2D(
            pool_size=merge_layer
        )
    )
    
    ml_layers.append(
        layers.Conv2D(
            64, 
            3, 
            padding='same', 
            activation='relu'
        )
    )
    
    ml_layers.append(
        layers.Conv2D(
            64, 
            3, 
            padding='same', 
            activation='relu'
        )
    )
    
    ml_layers.append(
        layers.Conv2D(
            64, 
            3, 
            padding='same', 
            activation='relu'
        )
    )
    
    ml_layers.append(
        layers.Conv2D(
            64, 
            3, 
            padding='same', 
            activation='relu'
        )
    )
    
    ml_layers.append(
        layers.MaxPooling2D(
            pool_size=merge_layer
        )
    )
    
    ml_layers.append(
        layers.Conv2D(
            64, 
            3, 
            padding='same', 
            activation='relu'
        )
    )
    
    ml_layers.append(
        layers.Conv2D(
            64, 
            3, 
            padding='same', 
            activation='relu'
        )
    )
    
    ml_layers.append(
        layers.Conv2D(
            64, 
            3, 
            padding='same', 
            activation='relu'
        )
    )
    
    ml_layers.append(
        layers.Conv2D(
            64, 
            3, 
            padding='same', 
            activation='relu'
        )
    )
    
    ml_layers.append(
        layers.MaxPooling2D(
            pool_size=merge_layer
        )
    )
    
    ml_layers.append(
        layers.Conv2D(
            32, 
            3, 
            padding='same', 
            activation='relu'
        )
    )
    
    ml_layers.append(
        layers.Conv2D(
            32, 
            3, 
            padding='same', 
            activation='relu'
        )
    )
    
    ml_layers.append(
        layers.Conv2D(
            32, 
            3, 
            padding='same', 
            activation='relu'
        )
    )
    
    ml_layers.append(
        layers.Conv2D(
            32, 
            3, 
            padding='same', 
            activation='relu'
        )
    )
    
    ml_layers.append(
        layers.MaxPooling2D(
            pool_size=merge_layer
        )
    )
    
    
    ml_layers.append(
        layers.Conv2D(
            16, 
            3, 
            padding='same', 
            activation='relu'
        )
    )
    
    ml_layers.append(
        layers.Conv2D(
            16, 
            3, 
            padding='same', 
            activation='relu'
        )
    )
    
    ml_layers.append(
        layers.Conv2D(
            16, 
            3, 
            padding='same', 
            activation='relu'
        )
    )
    
    ml_layers.append(
        layers.Conv2D(
            16, 
            3, 
            padding='same', 
            activation='relu'
        )
    )
    
    ml_layers.append(
        layers.MaxPooling2D(
            pool_size=merge_layer
        )
    )
    
    return ml_layers

In [13]:
def generate_output_layers(
    ml_layers: list
) -> list:
    
    ml_layers.append(
        layers.Flatten()
    )
    
    decision_neurons_size = get_size_of_training_set_categories() * 8
    
    ml_layers.append(
        layers.Dense(
            decision_neurons_size, 
            activation='relu',
        ),
    )
    
    ml_layers.append(
        layers.Dense(
            get_size_of_training_set_categories()
        )
    )
    
    return ml_layers

In [14]:
def generate_layers() -> list:
    layers = []
    
    layers = generate_input_layers(layers)
    layers = generate_middle_layers(layers)
    layers = generate_output_layers(layers)
    
    return layers

def make_model():
    global vision_size_w_channels
    
    division = (2, 2)
    
    return \
        Sequential(
            generate_layers()
        )

In [15]:
if not (strategy is None):
    with strategy.scope():
        model = make_model()
else:
    model = make_model()

In [16]:
model.compile(
    optimizer='adam', 
    loss=SparseCategoricalCrossentropy(
        from_logits=True
    ),
    metrics=['accuracy'], 
)
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 rescaling (Rescaling)       (None, 512, 512, 3)       0         
                                                                 
 conv2d (Conv2D)             (None, 512, 512, 256)     7168      
                                                                 
 conv2d_1 (Conv2D)           (None, 512, 512, 256)     590080    
                                                                 
 max_pooling2d (MaxPooling2D  (None, 256, 256, 256)    0         
 )                                                               
                                                                 
 conv2d_2 (Conv2D)           (None, 256, 256, 128)     295040    
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 128, 128, 128)    0         
 2D)                                                    

In [17]:
if use_cache:
    pointer_to_training_dataset = training_dataset.cache().prefetch(
        buffer_size=cache_buffer_size
    )
    
    pointer_to_validation_dataset = validation_dataset.cache().prefetch(
        buffer_size=cache_buffer_size
    )
else:
    pointer_to_training_dataset = training_dataset
    pointer_to_validation_dataset = validation_dataset

In [18]:
if get_log_runs():
    wandb_callback = WandbCallback(
        log_weights=True, 
        log_gradients=True, 
        training_data=pointer_to_training_dataset, 
        validation_data=pointer_to_validation_dataset,

        log_evaluation=True, 

        monitor='val_accuracy',
        mode='max'
    )

wandb: WARNING The save_model argument by default saves the model in the HDF5 format that cannot save custom objects like subclassed models and custom layers. This behavior will be deprecated in a future release in favor of the SavedModel format. Meanwhile, the HDF5 model is saved as W&B files and the SavedModel as W&B Artifacts.


In [19]:
if not (wandb_callback is None):
    fit_callbacks.append(wandb_callback)

In [20]:
history = None

In [21]:
if(len(fit_callbacks) > 0):
    history = model.fit(
        pointer_to_training_dataset,
        validation_data = pointer_to_validation_dataset,
        epochs=epochs,
        callbacks=fit_callbacks
    )
else:
    history = model.fit(
        pointer_to_training_dataset,
        validation_data = pointer_to_validation_dataset,
        epochs=epochs
    )

wandb: WARNING Error initializing ValidationDataLogger in WandbCallback. Skipping logging validation data. Error: '_BatchDataset' object is not subscriptable


Epoch 1/10
520/520 [==============================] - ETA: 0s - loss: 3.8921 - accuracy: 0.0475

wandb: Adding directory to artifact (/kaggle/working/wandb/run-20230614_160110-cl312tzo/files/model-best)... Done. 0.1s


520/520 [==============================] - 2238s 4s/step - loss: 3.8921 - accuracy: 0.0475 - val_loss: 3.8520 - val_accuracy: 0.0651
Epoch 2/10
520/520 [==============================] - 2143s 4s/step - loss: 3.8535 - accuracy: 0.0539 - val_loss: 3.8485 - val_accuracy: 0.0396
Epoch 3/10
520/520 [==============================] - 2156s 4s/step - loss: 3.8505 - accuracy: 0.0552 - val_loss: 3.8445 - val_accuracy: 0.0651
Epoch 4/10
520/520 [==============================] - 2165s 4s/step - loss: 3.8478 - accuracy: 0.0539 - val_loss: 3.8440 - val_accuracy: 0.0396
Epoch 5/10
520/520 [==============================] - 2144s 4s/step - loss: 3.8469 - accuracy: 0.0544 - val_loss: 3.8429 - val_accuracy: 0.0651
Epoch 6/10
520/520 [==============================] - 2149s 4s/step - loss: 3.8462 - accuracy: 0.0561 - val_loss: 3.8419 - val_accuracy: 0.0651
Epoch 7/10
520/520 [==============================] - 2115s 4s/step - loss: 3.8463 - accuracy: 0.0572 - val_loss: 3.8419 - val_accuracy: 0.0651
Epo

In [22]:
result = {
    'validation': {
        'accuracy': history.history['val_accuracy'],
        'loss': history.history['val_loss']
    },
    'result': {
        'accuracy': history.history['accuracy'],
        'loss': history.history['loss']
    }
}

results.append(result)

In [23]:
if (get_log_runs()):
    wandb.log( { 'training': results } )

model.save(path_to_model)

In [24]:
end_process()

wandb: Waiting for W&B process to finish... (success).
wandb: 
wandb: Run history:
wandb:     accuracy ▁▆▇▆▆▇████
wandb:        epoch ▁▂▃▃▄▅▆▆▇█
wandb:         loss █▂▂▂▁▁▁▁▁▁
wandb: val_accuracy █▁█▁██████
wandb:     val_loss █▆▄▃▃▂▂▁▂▁
wandb: 
wandb: Run summary:
wandb:          accuracy 0.05724
wandb:        best_epoch 0
wandb: best_val_accuracy 0.06507
wandb:             epoch 9
wandb:              loss 3.84344
wandb:      val_accuracy 0.06507
wandb:          val_loss 3.83966
wandb: 
wandb: 🚀 View run woven-jazz-20 at: https://wandb.ai/designermadsen/letter-identification/runs/cl312tzo
wandb: Synced 6 W&B file(s), 1 media file(s), 5 artifact file(s) and 1 other file(s)
wandb: Find logs at: ./wandb/run-20230614_160110-cl312tzo/logs
